## 1. Import all the packages/libraries first! <font color="#FF0000">(Execute Only)</font>

In [ ]:
#=========== RUN ME =========== 
import os
import re
import ast
import glob
import asyncio
import pandas as pd
import dask.dataframe as dd
import numpy as np
import swifter
import functools
import ipywidgets as widgets
# from yaspin import yaspin
from os import path
from os import listdir
from os import remove
# from threading import Timer
from collections import Counter
from collections import OrderedDict
from datetime import datetime
from tqdm import tqdm
# from rich.progress import track
# from alive_progress import alive_bar
from modin.config import ProgressBar
ProgressBar.enable()
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, \
HBox, VBox, Button, HTML, Tab, Output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

### 1.1 Set Global Variables for your project <font color="#FF0000">(Execute Only)</font>

In [ ]:
#Get absolute path instead of relative path
DIR = path.dirname(path.abspath("__file__"))
CSV_PATH = None
DEST_PATH = None
CFIDF_CSV_PATH = None
TFIDF_CSV_PATH = None
RANGE = 0

### 1.2 Initialize Widgets #Debounce & Main <font color="#FF0000">(Execute Only)</font>

In [ ]:
# Debounce
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator
    
# Folder Dropdown
foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))

def Dropdown(options, desc):
  return widgets.Dropdown(
    options=options,
    description=desc,
    disabled=False,
  )

# Folder Path Display Dropdown
folderPathLayout = widgets.Layout(width='auto')

def folderDisplay(desc):
  return widgets.Text(
    value='',
    placeholder='',
    description=desc,
    disabled=True,
    layout=folderPathLayout
)

# Button
def button(desc):
  return widgets.Button(description=desc)

## Bounded Integer
def boundedInt():
  return widgets.BoundedIntText(
    value=0,
    min=1,
    step=1,
    description=f'Position: ',
    disabled=False
  )

## Text
def inputText(desc):
  return widgets.Text(
    value='',
    placeholder='Type something here',
    description=desc,
    disabled=False
)

### 1.3 Initialize Widgets #Functions <font color="#FF0000">(Execute Only)</font>

In [ ]:
foldersOnly = list(filter(lambda x: path.isdir(x), listdir(DIR)))
#Widgets
folder_dropdown1 = Dropdown(foldersOnly, 'Folders: ')
# =====
file_dropdown1 = Dropdown([], 'Select Folder: ')

file_dropdown1.style = {'description_width': 'initial'}
# =====
folder_pathDisplay1 = folderDisplay('Folder Path: ')
# =====
input_Text = inputText('File name:')
# =====
btn_load1 = button('Load Source!')
btn_load2 = button('Load Previous!')
btn_load3 = button('Load Previous!')
btn_setDest = button('Set Path!')
btn_setRange = button('Set Range!')
btn_exeFx = button('Execute!')
btn_setFileName = button('Set!')

btn_setRange.style = {'description_width': 'initial'}
# =====
input_boundedInt = boundedInt()
# =====
# ===========================
# ===========================
# ===========================

# Functions
@debounce(0.5)
def changeFolder(change):
  folder_pathDisplay1.value = path.join(DIR, change['new'])
  file_dropdown1.options = listdir(folder_pathDisplay1.value)

# Functions for Reading Corpus
def loadDataset(btn):
  global CSV_PATH
  CSV_PATH = glob.glob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {CSV_PATH} as dataset source")

def loadTFIDFDataset(btn):
  global TFIDF_CSV_PATH
  TFIDF_CSV_PATH = glob.iglob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {TFIDF_CSV_PATH} as TFIDF dataset source")

def loadCFIDFDataset(btn):
  global CFIDF_CSV_PATH
  CFIDF_CSV_PATH = glob.iglob(path.join(folder_pathDisplay1.value, file_dropdown1.value, '*.csv'))
  print(f"Successfully set {CFIDF_CSV_PATH} as CSIDF dataset source")

def setDestination(btn):
  global RANGE, DEST_PATH
  RANGE = 1
  DEST_PATH = path.join(folder_pathDisplay1.value, file_dropdown1.value)
  print(f"Successfully set {DEST_PATH} as destination")
  print(f"Default file starting position is {RANGE}")

def setRange(btn):
  global RANGE
  RANGE = input_boundedInt.value
  print(f"Successfully set file starting position to {RANGE}")

folder_dropdown1.observe(changeFolder, 'value')

### 1.4 Initialize Widgets #Tabs <font color="#FF0000">(Execute Only)</font>

In [ ]:
tabTitles = ["Load Dataset", "Set File Name", "Select Column", "Load Previous Output", "Load Previous Output", "Set Output Destination", "Set Range", "Execute!"]
#Tab
tab = Tab()
#Tab Widgets
tab_load1 = VBox([
  widgets.HTML(value="<b><font style='color:red'>Select & Load your dataset Folder:</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load1
])

tab_setFileName = VBox([
  widgets.HTML(value="<b>Default output to main folder not assets!!</b>"),
  widgets.HTML(value="<b><font style='color:red'>Set output filename:</font></b>"),
  input_Text,
  btn_setFileName
])

tab_loadPrevious = VBox([
  widgets.HTML(value="Select & Load your previous output folder"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load2
])

tab_loadPrevious2 = VBox([
  widgets.HTML(value="Select & Load your previous output folder"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_load3
])

tab_setDest = VBox([
  widgets.HTML(value="Select Output Destination: <b><font style='color:red'>Make sure you have created a folder first!</font></b>"),
  folder_dropdown1,
  folder_pathDisplay1,
  file_dropdown1,
  btn_setDest
  ])

tab_setRange = VBox([
  widgets.HTML(value="Set the Starting Position for processing: <b>Default processing the <font style='color:red'>FIRST</font> file</b>"),
  input_boundedInt,
  btn_setRange
])

tab_execute = VBox([
  widgets.HTML(value='<b>If Everything is ready, Click the button below:</b>'),
  btn_exeFx
])